In [455]:
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
import numpy as np
import re
import pickle
import pandas as pd
import re

## Grab names for all NCAA teams

In [80]:
url = "https://www.sports-reference.com/cbb/schools/"
response = requests.get(url)
teams_text = response.text
team_soup = BeautifulSoup(teams_text, "lxml")
team_table = team_soup.find('table')
team_rows = team_table.find_all('tr')


In [250]:
#note that after every 20 teams, there is another header row
teams = {}
for i in [i for i in range(1,len(team_rows)) if i % 21 != 0]:
    items = team_rows[i].find_all('td')
    link = items[0].find('a')
    school, url = link.text, link['href']
    teams[school] = [url] + [i.text for i in items]

In [370]:
with open('teams_list.pickle', 'wb') as handle:
    pickle.dump(teams, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [381]:
#find teams in D1 for particular season
season = 2019
season_teams = []
for team in teams.keys():
    if int(teams[team][3]) <= season <= int(teams[team][4]):
        season_teams.append(team)

In [560]:
#create team_lookup for names - url name as key, values include name, formal name, and team url
team_lookup = {}
for key in season_teams:
    sched_url = "https://www.sports-reference.com" + teams[key][0] + str(season) +"-schedule.html"
    sched_response = requests.get(sched_url)
    sched_text = sched_response.text
    sched_soup = BeautifulSoup(sched_text,"lxml")
    sched_table = sched_soup.find('table')
    simple_name = re.split("\/",teams[key][0])[3]
    sched_rows = sched_table.find_all('tr')
    name = sched_table.find('a').text
    formal_name = teams[key][1]
    team_lookup[simple_name] = [name] + [formal_name] + [teams[key][0]]
with open('teams_lookup.pickle', 'wb') as handle:
    pickle.dump(team_lookup, handle, protocol=pickle.HIGHEST_PROTOCOL)

## GRAB gamelogs for each team for 2019

In [ ]:
gamelog = {}
for key in season_teams:
    game_url = "https://www.sports-reference.com" + teams[key][0] + str(season) +"-gamelogs.html"
    game_response = requests.get(game_url)
    gamelog_text = game_response.text
    gamelog_soup = BeautifulSoup(gamelog_text,"lxml")
    gamelog_table = gamelog_soup.find('table')
    gamelog_rows = gamelog_table.find_all('tr')
    team = re.split("\/",teams[key][0])[3]
    for i in [i for i in range(2,len(gamelog_rows)) if i % 22 != 0 and i % 23 != 0 ]:
        items = gamelog_rows[i].find_all('td')
        link = items[0].find('a')
        date, url = link.text, link['href']
        gamelog[url + team] = [url] + [team] + [i.text for i in items]

In [384]:
with open('gamelog.pickle', 'wb') as handle:
    pickle.dump(gamelog, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
advanced_gamelog = {}
for key in season_teams:
    game_url = "https://www.sports-reference.com" + teams[key][0] + str(season) +"-gamelogs-advanced.html"
    advanced_game_response = requests.get(game_url)
    advanced_gamelog_text = advanced_game_response.text
    advanced_gamelog_soup = advanced_BeautifulSoup(gamelog_text,"lxml")
    advanced_gamelog_table = advanced_gamelog_soup.find('table')
    advanced_gamelog_rows = advanced_gamelog_table.find_all('tr')
    team = re.split("\/",teams[key][0])[3]
    for i in [i for i in range(2,len(advanced_gamelog_rows)) if i % 22 != 0 and i % 23 != 0 ]:
        items = advanced_gamelog_rows[i].find_all('td')
        link = items[0].find('a')
        date, url = link.text, link['href']
        advanced_gamelog[url + team] = [url] + [team] + [i.text for i in items]

In [485]:
with open('advanced_gamelog.pickle', 'wb') as handle:
    pickle.dump(advanced_gamelog, handle, protocol=pickle.HIGHEST_PROTOCOL)

## GRAB URL FOR ALL GAMES

## Grab data from single game - hold off on pulling individual game data for now

Input url for each game
Output data table

In [7]:
url = 'https://www.sports-reference.com/cbb/boxscores/2020-03-11-21-stanford.html'

box = requests.get(url)
boxscores = box.text
soup = BeautifulSoup(boxscores, "html5lib")

In [65]:
tables = soup.find_all('table')
#note this will grab 8 tables (the last 4 will be basic and advanced stats, for each team)

Pull Data from class "scorebox_meta": Date, Location <- this does not show up in a table

Pull Data from table "line-score": <- this also does not show up in a table...
Output: Away Team, Home Team, Away Score 1H, Away Score 2H, Away Score F, Home Score 1H, Home Score 2h, Home Score F

? What about overtimes...

## Scratchwork

In [583]:
#test case for gamelogs
cal_url = "https://www.sports-reference.com/cbb/schools/california/2019-gamelogs.html"
cal_response = requests.get(cal_url)
gamelog_text = cal_response.text
gamelog_soup = BeautifulSoup(gamelog_text,"lxml")
gamelog_table = gamelog_soup.find('table')
gamelog_rows = gamelog_table.find_all('tr')
#test case continued
cal_log = {}
team = 'california'
for i in [i for i in range(2,len(gamelog_rows)) if i % 22 != 0 and i % 23 != 0 ]:
    items = gamelog_rows[i].find_all('td')
    link = items[0].find('a')
    date, url = link.text, link['href']
    cal_log[url + team] = [url] + [team] + [i.text for i in items]